## 임베딩 레이어 추출

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn


path = os.path.dirname(os.path.abspath("tokenizer.ipynb"))
model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B"


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=path)

In [ ]:
model = AutoModel.from_pretrained(model_name)

In [ ]:
embedding_layer = model.get_input_embeddings()

# 임베딩 레이어를 추출한 후 이를 별도의 nn.Module로 래핑합니다.
class EmbeddingsOnlyModel(nn.Module):
    def __init__(self, embedding_layer):
        super(EmbeddingsOnlyModel, self).__init__()
        self.embeddings = embedding_layer

    def forward(self, input_ids):
        return self.embeddings(input_ids)

# 래핑된 모델 생성
embeddings_only_model = EmbeddingsOnlyModel(embedding_layer)

# 래핑된 모델을 저장합니다.
model_path = "embeddings_only_model.pth"
torch.save(embeddings_only_model.state_dict(), model_path)

## 임베딩 레이어 로딩

In [ ]:
import torch
from transformers import AutoTokenizer
import torch.nn as nn

# Llama3 모델의 토크나이저를 불러옵니다.
model_name = "MLP-KTLim/llama-3-Korean-Bllossom-8B"  # 모델 이름을 Llama3 모델로 대체하세요.
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 임베딩 레이어를 정의합니다.
embedding_layer = nn.Embedding.from_pretrained(torch.empty((128256, 4096)))  # 크기는 예시입니다.

# 임베딩 레이어를 추출한 후 이를 별도의 nn.Module로 래핑합니다.
class EmbeddingsOnlyModel(nn.Module):
    def __init__(self, embedding_layer):
        super(EmbeddingsOnlyModel, self).__init__()
        self.embeddings = embedding_layer

    def forward(self, input_ids):
        return self.embeddings(input_ids)

# 래핑된 모델 인스턴스를 생성합니다.
embeddings_only_model = EmbeddingsOnlyModel(embedding_layer)

# 저장된 모델 상태를 불러옵니다.
model_path = "embeddings_only_model.pth"
embeddings_only_model.load_state_dict(torch.load(model_path))
print("Wrapped model loaded")

# 테스트: 토크나이저를 사용하여 입력 텍스트를 토큰화하고, 임베딩 레이어를 통해 임베딩을 계산합니다.
input_text = "안녕 얘들아!! 오랬만이야"
input_ids = tokenizer(input_text, return_tensors="pt")["input_ids"]
embeddings = embeddings_only_model(input_ids)

embeddings